In [1]:
from elliot.run import run_experiment


__/\\\\\\\\\\\\\\\___/\\\\\\______/\\\\\\_________________________________________        
 _\/\\\///////////___\////\\\_____\////\\\_________________________________________       
  _\/\\\_________________\/\\\________\/\\\______/\\\_____________________/\\\______      
   _\/\\\\\\\\\\\_________\/\\\________\/\\\_____\///_______/\\\\\______/\\\\\\\\\\\_     
    _\/\\\///////__________\/\\\________\/\\\______/\\\____/\\\///\\\___\////\\\////__    
     _\/\\\_________________\/\\\________\/\\\_____\/\\\___/\\\__\//\\\_____\/\\\______   
      _\/\\\_________________\/\\\________\/\\\_____\/\\\__\//\\\__/\\\______\/\\\_/\\__  
       _\/\\\\\\\\\\\\\\\___/\\\\\\\\\___/\\\\\\\\\__\/\\\___\///\\\\\/_______\//\\\\\___ 
        _\///////////////___\/////////___\/////////___\///______\/////__________\/////____
Version Number: 0.3.1


In [2]:
# run_experiment("configuration/file/path")

In [3]:
# Had to reduce tf version to 2.2.0 - it may cause compatibility problems.
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
    ]
)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])

model.fit(x_train, y_train, epochs=5)

2023-02-28 18:59:07.642501: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-02-28 18:59:07.974949: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-28 18:59:07.975004: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 Laptop GPU computeCapability: 8.6
coreClock: 1.702GHz coreCount: 30 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 312.97GiB/s
2023-02-28 18:59:07.975175: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-02-28 18:59:07.977897: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-28 18:59:07.979031: I tensorflow/str

Epoch 1/5


2023-02-28 19:00:32.165533: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


1875/1875 [==============================] - 4s 2ms/step - loss: 0.2985 - accuracy: 0.9134
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1433 - accuracy: 0.9576
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1077 - accuracy: 0.9671
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0874 - accuracy: 0.9731
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0755 - accuracy: 0.9761


In [4]:
tf.config.list_physical_devices(device_type='GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

Num GPUs Available:  1


2023-02-28 19:02:33.489716: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14052370507845418453,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 11474872544011995249
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13235327312586754945
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 5061371488
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11864126033979055280
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"]

2023-02-28 19:02:33.489780: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 Laptop GPU computeCapability: 8.6
coreClock: 1.702GHz coreCount: 30 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 312.97GiB/s
2023-02-28 19:02:33.489839: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2023-02-28 19:02:33.489855: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2023-02-28 19:02:33.489867: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2023-02-28 19:02:33.489879: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2023-02-28 19:02:33.489890: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic lib